In [ ]:
!pip install -q transformers accelerate datasets[audio]
#!pip install -q flash-attn --no-build-isolation # we are using optimum BetterTransformer since Flash Attention 2 isn't supported on Colab
!pip install --q optimum


In [ ]:
import textwrap

wrapper = textwrap.TextWrapper(width=80,
    initial_indent=" " * 8,
    subsequent_indent=" " * 8,
    break_long_words=False,
    break_on_hyphens=False)

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from optimum.bettertransformer import BetterTransformer


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [ ]:
model_id = "distil-whisper/distil-medium.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True#, use_flash_attention_2=True
)
model.to(device)
model = model.to_bettertransformer() # we are using optimum BetterTransformer since Flash Attention 2 isn't supported on Colab

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15, #long form transcription
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
from google.colab import files
files.upload()

In [ ]:
result = pipe('your-file-goes-here.mp3')
#print(result["text"])


In [ ]:
print(wrapper.fill(result["text"]))